Привет! меня зовут Люман Аблаев. Сегодня я проверю твой проект.
<br> Дальнейшее общение будет происходить на "ты" если это не вызывает никаких проблем.
<br> Желательно реагировать на красные комментарии ('исправил', 'не понятно как исправить ошибку', ...)
<br> Пожалуйста, не удаляй комментарии ревьюера, так как они повышают качество повторного ревью.

Комментарии будут в <font color='green'>зеленой</font>, <font color='blue'>синей</font> или <font color='red'>красной</font> рамках:


<div class="alert alert-block alert-success">
<b>Успех:</b> Если все сделано отлично
</div>

<div class="alert alert-block alert-info">
<b>Совет: </b> Если можно немного улучшить
</div>

<div class="alert alert-block alert-danger">
<b>Ошибка:</b> Если требуются исправления. Работа не может быть принята с красными комментариями.
</div>

-------------------

Будет очень хорошо, если ты будешь помечать свои действия следующим образом:
<div class="alert alert-block alert-warning">
<b>Комментарий студента:</b> ..
</div>
<div class="alert alert-block alert-warning">
<b>Изменения:</b> Были внесены следующие изменения ...
</div>

<font color='orange' style='font-size:24px; font-weight:bold'>Общее впечатление</font>
* Спасибо за  качественную работу, было интересно ее проверять.
- Я оставил некоторые советы надеюсь они будут полезными или интересными.

- Есть небольшие недочеты, но думаю у тебя не составить труда их исправить
- Жду обновленную работу!

<font color='orange' style='font-size:24px; font-weight:bold'>Общее впечатление[2]</font>
* Спасибо за оперативность
- Недочеты исправлены - работа полностью корректна
- Рад был быть полезен. Удачи в дальнейшем обучении!


# Выбор локации для скважины

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

In [1]:
#pandas для работы с датафреймами
import pandas as pd
#математические библиотеки
import math
import numpy as np
#графики
from scipy import stats as st
import matplotlib.pyplot as plt
import seaborn as sns
#sklearn
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler 

from sklearn.metrics import (
    mean_squared_error,
    r2_score
)

from sklearn.pipeline import Pipeline

from sklearn.impute import SimpleImputer

Введём константы.

In [2]:
RANDOM_STATE = 42
TEST_SIZE = 0.25



<div class="alert alert-block alert-success">
<b>Успех:</b> Импорты и настройки на месте
</div>


## Загрузка и подготовка данных

Создадим список, куда добавим датафреймы.

In [3]:
data = list()

### Первый регион

Загрузим данные для первого региона.

In [4]:
data.insert(0, pd.read_csv('/datasets/geo_data_0.csv'))
data[0].head()

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


Проверим методом info.

In [5]:
data[0].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


Найдём дубликаты.

In [6]:
data[0].duplicated().sum()

0

Явных дубликатов нет.

Найдём дубликаты в столбце id.

In [7]:
data[0].duplicated(subset=['id']).sum()

10

Проанализируем значения столбцов у всех дубликатов.

In [8]:
data[0][data[0].duplicated(subset=['id'], keep=False)].sort_values(by='id')

,id,f0,f1,f2,product
66136,74z30,1.084962,-0.312358,6.990771,127.643327
64022,74z30,0.741456,0.459229,5.153109,140.771492
51970,A5aEY,-0.180335,0.935548,-2.094773,33.020205
3389,A5aEY,-0.039949,0.156872,0.209861,89.249364
69163,AGS9W,-0.933795,0.116194,-3.655896,19.230453
42529,AGS9W,1.454747,-0.479651,0.683380,126.370504
931,HZww2,0.755284,0.368511,1.863211,30.681774
7530,HZww2,1.061194,-0.373969,10.430210,158.828695
63593,QcMuo,0.635635,-0.473422,0.862670,64.578675
1949,QcMuo,0.506563,-0.323775,-2.215583,75.496502


Значения f0, f1, f2 и product сильно различаются для каждого дубликата, невозможно понять какие именно значения истинные. Таких столбцов всего 20, это мало на фоне 100000 скважен, следовательно от дубликатов можно избавиться.

In [9]:
data[0] = data[0].drop_duplicates(subset=['id'], keep=False).reset_index(drop=True)
data[0].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99980 entries, 0 to 99979
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   id       99980 non-null  object 
 1   f0       99980 non-null  float64
 2   f1       99980 non-null  float64
 3   f2       99980 non-null  float64
 4   product  99980 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


f0, f1 и f2 придётся масштабировать, аномалий не обнаружено.

### Второй регион

Загрузим данные.

In [10]:
data.insert(1, pd.read_csv('/datasets/geo_data_1.csv'))
data[1].head()

,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


Рассмотрим методом info.

In [11]:
data[1].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


Проверим на дубликаты.

In [12]:
data[1].duplicated().sum()

0

Явных дубликатов нет.

Проверим на дубликаты в столбце id

In [13]:
data[1].duplicated(subset=['id']).sum()

4

Поступим с дубликатами аналогично действиям выше.

In [14]:
data[1][data[1].duplicated(subset=['id'], keep=False)].sort_values(by='id')

,id,f0,f1,f2,product
5849,5ltQ6,-3.435401,-12.296043,1.999796,57.085625
84461,5ltQ6,18.213839,2.191999,3.993869,107.813044
1305,LHZR0,11.170835,-1.945066,3.002872,80.859783
41906,LHZR0,-8.989672,-4.286607,2.009139,57.085625
2721,bfPNe,-9.494442,-5.463692,4.006042,110.992147
82178,bfPNe,-6.202799,-4.820045,2.995107,84.038886
47591,wt4Uk,-9.091098,-8.109279,-0.002314,3.179103
82873,wt4Uk,10.259972,-9.376355,4.994297,134.766305


Дубликатов относительно всей таблицы мало, избавимся от них.

In [15]:
data[1] = data[1].drop_duplicates(subset=['id'], keep=False).reset_index(drop=True)
data[1].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99992 entries, 0 to 99991
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   id       99992 non-null  object 
 1   f0       99992 non-null  float64
 2   f1       99992 non-null  float64
 3   f2       99992 non-null  float64
 4   product  99992 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


### Третий регион

Загрузим данные.

In [16]:
data.insert(2, pd.read_csv('/datasets/geo_data_2.csv'))
data[2].head()

,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


Рассмотрим методом info.

In [17]:
data[2].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


Проверим на дубликаты.

In [18]:
data[2].duplicated().sum()

0

Явных дубликатов нет.

Проверим в столбце id.

In [19]:
data[2].duplicated(subset=['id']).sum()

4

In [20]:
data[2][data[2].duplicated(subset=['id'], keep=False)].sort_values(by='id')

,id,f0,f1,f2,product
45404,KUPhW,0.231846,-1.698941,4.990775,11.716299
55967,KUPhW,1.211150,3.176408,5.543540,132.831802
11449,VF7Jo,2.122656,-0.858275,5.746001,181.716817
49564,VF7Jo,-0.883115,0.560537,0.723601,136.233420
44378,Vcm5J,-1.229484,-2.439204,1.222909,137.968290
95090,Vcm5J,2.587702,1.986875,2.482245,92.327572
28039,xCHr8,1.633027,0.368135,-2.378367,6.120525
43233,xCHr8,-0.847066,2.101796,5.597130,184.388641


Избавимся от дубликатов.

In [21]:
data[2] = data[2].drop_duplicates(subset=['id'], keep=False).reset_index(drop=True)
data[2].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99992 entries, 0 to 99991
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   id       99992 non-null  object 
 1   f0       99992 non-null  float64
 2   f1       99992 non-null  float64
 3   f2       99992 non-null  float64
 4   product  99992 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


Построим гистограммы.

Аномалии на гистограммах не обнаружены, значения f0, f1 и f2 необходимо масштабировать.


<div class="alert alert-block alert-success">
<b>Успех:</b> Данные осмотрены, но можно было бы еще как минимум изучить корреляции, так как мы планируем использовать линейные модели, а для них это важно. 
</div>
<div class="alert alert-block alert-info">
<b>Совет:</b>  Советую обратить внимание на библиотеки sweetviz и ydata_profiling помогут в проведении более тщательного EDA анализа. Исследовательский анализ можно делать и с помощью ручного вызова функций дефолтных библиотек. Данные библиотеки хороши для максимизации комфорта презентации результатов анализа бизнес-пользователям.  

Очень просты в использоовании, на вход кладется датафрейм: ydata_profiling.ProfileReport(df) 
</div>

## Обучение и проверка модели

Создадим функцию для масштабирования

Напишем цикл для обучение моделей на трёх датафреймах.

In [22]:
models = list() #список с моделями линейной регрессии
y_true_valid = list() #список с правильными ответами на валидационной выборке
predicts = list() #список с предсказаниями на валидационной выборке
rmse = list() #список с RMSE моделей

for i, data_i in enumerate(data):
    
    #разделим выборку на тренировочную и валидационную
    X_train, X_valid, y_train, y_valid = train_test_split(
        data_i.drop(['id', 'product'], axis=1),
        data_i['product'],
        test_size=TEST_SIZE,
        random_state=RANDOM_STATE
    )
    
    y_train = y_train.reset_index(drop=True)
    y_true_valid.insert(i, y_valid.reset_index(drop=True))
    
    #масштабируем количественные признаки
    scaler = StandardScaler()
    
    X_train_scaled = scaler.fit_transform(X_train)
    X_valid_scaled = scaler.transform(X_valid)

    X_train_scaled = pd.DataFrame(X_train_scaled, columns=['f0', 'f1', 'f2'])
    X_valid_scaled = pd.DataFrame(X_valid_scaled, columns=['f0', 'f1', 'f2'])
    
    #создадим модель и обучим
    models.insert(i, LinearRegression())
    
    models[i].fit(X_train_scaled, y_train)
    
    #сделаем предсказание
    predicts.insert(i, pd.Series(models[i].predict(X_valid_scaled)))
    
    #найдём RMSE
    rmse.insert(i, mean_squared_error(y_true_valid[i], predicts[i], squared=False))

Выпишем среднее и RMSE для каждой модели.

In [23]:
for i in range(3):
    print(f'Среднее для региона {i+1} равняется {predicts[i].mean()}, дисперсия {rmse[i]}, {y_true_valid[i].mean()} \n')

Среднее для региона 1 равняется 92.2865385223536, дисперсия 37.76114714593895, 92.37491536925566 

Среднее для региона 2 равняется 69.16894101447984, дисперсия 0.892863589877732, 69.16493608452177 

Среднее для региона 3 равняется 94.82069136328788, дисперсия 40.13115560413051, 95.05858294595217 




<div class="alert alert-block alert-success">
<b>Успех:</b> Шаг проделан лаконично, все метрики посчитаны корректно
</div> 

## Подготовка к расчёту прибыли

Рассчитаем минимальный запас сырья для безубыточной работы. Для начала создадим переменные с бюджетом и ценой.

In [24]:
BUDGET = 10 ** 10 #бюджет
COST = 450 * 10 ** 3 #цена за 1000 баррелей


<div class="alert alert-block alert-info">
    
<b>Совет:</b> Константы принято записывать в ВЕРХНЕМ РЕГИСТРЕ =) Читем здесь https://pythonworld.ru/osnovy/pep-8-rukovodstvo-po-napisaniyu-koda-na-python.html#section-29
    

Чтобы найти минимальный объём сырья для безубыточной работы скважины разделим бюджет на цену за 1000 баррелей, умноженной на 200, так как будут создаваться 200 скважин.

In [25]:
MIN_VOLUME = BUDGET / (COST * 200)
MIN_VOLUME

111.11111111111111

Минимальный объём скважины больше чем среднее для любого из регионов, однако максимальный может быть больше.

<div class="alert alert-block alert-success">
<b>Успех:</b> Значение для безубыточной разработки посчитано верно, с выводом согласен  
</div>

## Расчёт прибыли и рисков 

Необходимо отобрать 200 лучших скважин в каждом регионе и найти прибыль от этих скважин. Для решения задачи напишем функцию.

In [34]:
predicts[0]

0        140.415284
1         84.683193
2        138.812581
3        117.496790
4         75.558549
            ...    
24990     61.107288
24991    120.638882
24992    100.291635
24993     70.973992
24994     46.790570
Length: 24995, dtype: float64

In [37]:
y_true_valid[0].index

RangeIndex(start=0, stop=24995, step=1)

In [41]:
predicts[0].sort_values(ascending=False)[:200].index

Int64Index([  305, 21620,   752, 12584,  6216, 15540, 24818, 18323,  2550,
            15744,
            ...
            14332, 21643, 16920, 23617, 22439,  8198,  7498, 12373,  3596,
            14300],
           dtype='int64', length=200)

In [42]:
y_true_valid[0][predicts[0].sort_values(ascending=False)[:200].index]

305      144.009283
21620    162.810993
752      123.920559
12584    155.212534
6216     145.106893
            ...    
8198     106.762173
7498     140.870781
12373    160.082468
3596     130.539586
14300    102.618513
Name: product, Length: 200, dtype: float64

In [28]:
def get_profit(pred):
    return pred.sum() * COST - BUDGET

<div class="alert alert-block alert-danger">
<b>Ошибка:</b> Для выбора 200 лучших скважин нужно использовать предсказанные значения. А вот для подсчета прибыли должны использоваться настоящие значения соответствующих ячеек.

</div>

<div class="alert alert-block alert-success">
<b>Успех[2]:</b> Есть контакт
</div>


Создадим цикл для прибыли.

In [44]:
for i in range(3):
    print(
        f'Прибыль для региона {i+1} составляет ' 
        f'{get_profit(y_true_valid[i][predicts[i].sort_values(ascending=False)[:200].index])}'
    )

Прибыль для региона 1 составляет 3301821343.6462536
Прибыль для региона 2 составляет 2415086696.681511
Прибыль для региона 3 составляет 2103914701.2991962


Прибыль показана для скважин с максимальным значением сырья.

Найдём распределение прибыли с помощью метода bootstrap, для чего отберём 500 случайных скважин и найдём прибыль с 200 лучшими.

Функция для бутстрапа.

In [51]:
def bootstrap(pred, valid):
    profit = list()
    
    for i in range(1000):
        index = pred.sample(n=500, replace=True).sort_values(ascending=False)[:200].index
        profit.insert(i,get_profit(valid[index]))
        
    return pd.Series(profit)


<div class="alert alert-block alert-success">
<b>Успех:</b> С бутстрапом тоже все в порядке (сэмплирование происходит с возвращением)
</div>


Найдём распределение прибыли для бутстрапа для каждого региона

In [52]:
profit_dist = list()

for i in range(3):
    profit_dist.insert(i, bootstrap(predicts[i], y_true_valid[i]))

Найдём среднюю прибыль, 95% доверительного интервала и риск убытков.

Функция для процента убытков.

In [53]:
def risk(data):
    return data[data<0].count() / len(data)

Выведем среднее, квантили и процент убытков для каждого региона.

In [54]:
for i in range(3):
    print(f'Регион {i+1}:')
    print(f'Средняя прибыль {profit_dist[i].mean()}')
    print(f'Доверительный интервал 95% от {profit_dist[i].quantile(0.025)} до {profit_dist[i].quantile(0.975)}')
    print(f'Убытки: {risk(profit_dist[i])} \n')

Регион 1:
Средняя прибыль 386243015.9923171
Доверительный интервал 95% от -139293281.56638464 до 921439844.8228406
Убытки: 0.073 

Регион 2:
Средняя прибыль 473454030.4874329
Доверительный интервал 95% от 70320926.48706298 до 847980560.4245868
Убытки: 0.01 

Регион 3:
Средняя прибыль 364337796.9504442
Доверительный интервал 95% от -158150637.4817801 до 854999898.1968027
Убытки: 0.079 



<div class="alert alert-block alert-success">
<b>Успех:</b> метрики считаются верным образом!  
</div>


## Выводы

На основе данных выше, получается, что максимальная прибыль будет от второго региона, там самое большое среднее и меньше всего вероятность убытков.

<div class="alert alert-block alert-info">
<b>Совет: </b>
    
Не забудь обновить выводы, если потребуется
</div>                                                              



<div class="alert alert-block alert-success">
<b>Успех[2]:</b> Отлично, проведено детальное исследование и был выбран правильный регион! Но не стесняйся делать выводы содержательнее
</div>



<div class="alert alert-block alert-info">
<b>Совет[2]: </b> Если интересно изучить тему бутстрапа глубже - я оставлю  ниже несколько интересных и полезных ссылок по теме:
        
1. "Bootstrap Hypothesis Testing in Statistics with Example" 
        
https://www.youtube.com/watch?v=9STZ7MxkNVg
        
2. "How to Calculate Bootstrap Confidence Intervals For Machine Learning Results in Python" 
        
https://machinelearningmastery.com/calculate-bootstrap-confidence-intervals-machine-learning-results-python/
        
3. "A Gentle Introduction to the Bootstrap Method" 

https://machinelearningmastery.com/a-gentle-introduction-to-the-bootstrap-method/
        
4. "An Introduction to the Bootstrap Method" 
        
https://towardsdatascience.com/an-introduction-to-the-bootstrap-method-58bcb51b4d60#:~:text=The%20basic%20idea%20of%20bootstrap,population%20mean)%20on%20sample%20data.&amp;text=A%20sample%20from%20population%20with%20sample%20size%20n
        
5. "Bootstrapping and Resampling in Statistics with Example" 
        
        
https://www.youtube.com/watch?v=O_Fj4q8lgmc

6. Также одна из программ на рынке по этой теме управлненеческого моделирования - AnyLogic - интересный продукт. Можешь видео из интернета посмотреть по какому-нибудь примеру с её использованием. Тогда тема управленческого моделирования и бутстрапа может по-другому лечь.
</div>

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  Jupyter Notebook открыт
- [ ]  Весь код выполняется без ошибок
- [ ]  Ячейки с кодом расположены в порядке исполнения
- [ ]  Выполнен шаг 1: данные подготовлены
- [ ]  Выполнен шаг 2: модели обучены и проверены
    - [ ]  Данные корректно разбиты на обучающую и валидационную выборки
    - [ ]  Модели обучены, предсказания сделаны
    - [ ]  Предсказания и правильные ответы на валидационной выборке сохранены
    - [ ]  На экране напечатаны результаты
    - [ ]  Сделаны выводы
- [ ]  Выполнен шаг 3: проведена подготовка к расчёту прибыли
    - [ ]  Для всех ключевых значений созданы константы Python
    - [ ]  Посчитано минимальное среднее количество продукта в месторождениях региона, достаточное для разработки
    - [ ]  По предыдущему пункту сделаны выводы
    - [ ]  Написана функция расчёта прибыли
- [ ]  Выполнен шаг 4: посчитаны риски и прибыль
    - [ ]  Проведена процедура *Bootstrap*
    - [ ]  Все параметры бутстрепа соответствуют условию
    - [ ]  Найдены все нужные величины
    - [ ]  Предложен регион для разработки месторождения
    - [ ]  Выбор региона обоснован